In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
from tqdm import tqdm

import os

ROOT = r'L:\\basic\\divg\\PATH-COMPAI\\datasets\\Barrett\\Barrett ASL 21-11-22\\'

In [ ]:
data_path = os.path.join(ROOT, "patch_dataset_s1_512", "p53")
patch_paths = [os.path.join(data_path, nm) for nm in os.listdir(data_path)]

size = cv2.imread(patch_paths[0]).shape[0]

brown = np.array([128,  90,  90])
blue  = np.array([150, 170, 190])
template = np.ones((size*size,3))
brown_copy = template.copy()
blue_copy  = template.copy()
brown_copy[:] = brown
blue_copy [:] = blue

for i, path in tqdm(enumerate(patch_paths)):
    try:
        img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    except:
        print(path)
        continue
    pixels = img.reshape(size*size,3)
    brown_mask = np.isclose(pixels, brown_copy, atol=50).astype(np.float32).min(axis=-1).reshape(size,size)
    blue_mask  = np.isclose(pixels, blue_copy , atol=50).astype(np.float32).min(axis=-1).reshape(size,size)

    fig, ax = plt.subplots(1,2)
    ax[0].imshow(img)
    ax[0].axis("Off")
    ax[1].imshow(brown_mask, cmap="gray")
    ax[1].axis("Off")
    ax[1].set_title(f"{brown_mask.sum()/(blue_mask.sum()+brown_mask.sum()):.4f}")

    plt.savefig(os.path.join(ROOT, "visualisation", os.path.basename(path)), bbox_inches="tight", 
                # pad_inches=0
                )
    plt.close()

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(30,30))
ax[0].imshow(img)
ax[1].imshow(brown_mask)
ax[2].imshow(blue_mask)

In [ ]:
img2.sum()/img3.sum()